# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../WeatherPy/cities.csv')
cities_df

# delete extraneous column
del cities_df['city_id']
cities_df

# drop cities with missing info
cities_df = cities_df.dropna(how='any')

# check to see if df complete
cities_df.count()

cities_df.head()

,city,cloudiness,country,date,humidity,lat,lng,maxtemp,windspeed
0,Faanui,17,PF,1579884368,74.0,-16.48,-151.75,28.00,6.50
1,Jamestown,90,US,1579884368,41.0,42.10,-79.24,7.22,5.10
2,Vaini,40,TO,1579884368,83.0,-21.20,-175.20,25.00,3.60
3,Chopovychi,99,UA,1579884368,73.0,50.83,28.95,0.41,6.35
4,Port Hueneme,1,US,1579884328,66.0,34.15,-119.20,20.56,1.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# configre gmaps
gmaps.configure(api_key=gkey)

# store lat and long in locations
locations = cities_df[['lat','lng']]

# store humidity to use as weight
humidity = cities_df['humidity'].astype('float')

In [19]:
# plot heatmap
fig = gmaps.figure()

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
                                 #dissipating=False, max_intensity=100, point_radius=1)

# add layer to map
fig.add_layer(heat_layer)

# display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [88]:
print(cities_df.dtypes)

# narrow df to ideal conditions
ideal_cities_df = cities_df.loc[(cities_df['humidity']<80) & 
                                (cities_df['maxtemp']>23) & 
                                (cities_df['maxtemp']<30) &
                                (cities_df['cloudiness'].astype('float')==0) &
                                (cities_df['windspeed']<4)]
ideal_cities_df

city           object
cloudiness     object
country        object
date            int64
humidity      float64
lat           float64
lng           float64
maxtemp       float64
windspeed     float64
dtype: object


,city,cloudiness,country,date,humidity,lat,lng,maxtemp,windspeed
44,Kavieng,0,PG,1579884372,65.0,-2.57,150.80,28.49,2.67
51,Bredasdorp,0,ZA,1579884373,73.0,-34.53,20.04,23.33,1.00
68,Chui,0,UY,1579884375,44.0,-33.70,-53.46,27.83,2.95
224,Mumford,0,GH,1579884392,65.0,5.26,-0.76,28.78,3.98
246,San Vicente,0,AR,1579884394,76.0,-26.62,-54.13,26.11,0.45
256,Saldanha,0,ZA,1579884395,54.0,-33.01,17.94,26.00,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [97]:
hotels_df=ideal_cities_df
hotels_df['hotel name']=""
hotels_df

/Users/lorelley/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,cloudiness,country,date,humidity,lat,lng,maxtemp,windspeed,hotel name
44,Kavieng,0,PG,1579884372,65.0,-2.57,150.80,28.49,2.67,
51,Bredasdorp,0,ZA,1579884373,73.0,-34.53,20.04,23.33,1.00,
68,Chui,0,UY,1579884375,44.0,-33.70,-53.46,27.83,2.95,
224,Mumford,0,GH,1579884392,65.0,5.26,-0.76,28.78,3.98,
246,San Vicente,0,AR,1579884394,76.0,-26.62,-54.13,26.11,0.45,
256,Saldanha,0,ZA,1579884395,54.0,-33.01,17.94,26.00,3.60,


In [106]:
radius = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "Hotels",
    "radius": radius,
    "key": gkey,
    }

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    params['location']=f'{lat},{lng}'
    
    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    
    try:
        hotels_df.loc[index, "hotel name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df


,city,cloudiness,country,date,humidity,lat,lng,maxtemp,windspeed,hotel name
44,Kavieng,0,PG,1579884372,65.0,-2.57,150.80,28.49,2.67,Kaewieng
51,Bredasdorp,0,ZA,1579884373,73.0,-34.53,20.04,23.33,1.00,Bredasdorp
68,Chui,0,UY,1579884375,44.0,-33.70,-53.46,27.83,2.95,Chuy
224,Mumford,0,GH,1579884392,65.0,5.26,-0.76,28.78,3.98,Apam
246,San Vicente,0,AR,1579884394,76.0,-26.62,-54.13,26.11,0.45,San Pedro
256,Saldanha,0,ZA,1579884395,54.0,-33.01,17.94,26.00,3.60,Saldanha


In [101]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [105]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))